In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview
In this notebook, we will modify the imported DialogFlow CX Agent for CymBuddy. We will update the Webhooks to use the URI of the deployed Cloud Functions.

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it.
- `output_urls.txt` file which contains the URIs of the deployed Cloud Functions. This file will be generated automatically and should be present after deploying resources through Terraform.


{TODO: Update the links}
<table align="left">
  <td style="text-align: center">
    <a href="">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

In [2]:
!pip install dfcx-scrapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.9 MB/s eta 0:00:00
  Created wheel for dfcx-scrapi: filename=dfcx_scrapi-1.10.5-py3-none-any.whl size=203247 sha256=f3d7b35191386ed7c555e8d2fea9a93be21091a063aad75b9679504656bd1068
  Stored in directory: /root/.cache/pip/wheels/0c/63/58/859bf913596be3b865980a7bcf8e0c1d0f694665ebc8776fa0
Successfully built dfcx-scrapi


# Imports


In [1]:
from dfcx_scrapi.tools.copy_util import CopyUtil
from google.cloud import dialogflowcx_v3beta1 as dialogflow

# User Inputs
In the next section, we will collect runtime variables needed to execute this notebook.   
This should be the only cell of the notebook you need to edit in order for this notebook to run.

For this notebook, we'll need the following inputs:

- `creds_path`: Your local path to your GCP Service Account Credentials
- `agent_id`: Your Dialogflow CX Agent ID in String format

In [5]:
creds_path = "{YOUR_CREDS_FILE_PATH}"  # @param {type:"string"}

# should be of the format: projects/<Project ID>/locations/<Location ID>/agents/<Agent ID>
CymBuddy_agent_id = "{YOUR-AGENT-ID}"  # @param {type:"string"}

# Get Webhooks Map from Agent
First, we will extract a `map` of the Webhook IDs and Webhook Display Names from the Agent.   
We pass `reverse=True` to the `get_webhooks_map` function which provides the Display Names as keys and the IDs as values.   
This allows us to refernce the map using Display Names vs. the long, cumbersome ID names.

In [7]:
cu = CopyUtil(creds_path)

webhooks_map = cu.webhooks.get_webhooks_map(CymBuddy_agent_id, reverse=True)
print(webhooks_map)

{'Is the destination in India?': 'projects/terraform-test-project-406510/locations/us-central1/agents/e92ba26a-098c-4f5f-96d6-1f9832e30fc7/webhooks/03153ec6-a908-4ccc-b8dc-684e5feddc7f', 'Event_Recommendation_Travelling': 'projects/terraform-test-project-406510/locations/us-central1/agents/e92ba26a-098c-4f5f-96d6-1f9832e30fc7/webhooks/0317fed1-a3c1-4d76-aa9e-e5f30df3b2ac', 'Extend Overdraft': 'projects/terraform-test-project-406510/locations/us-central1/agents/e92ba26a-098c-4f5f-96d6-1f9832e30fc7/webhooks/08d9e0d2-6a4f-421f-8135-15b5d3313867', 'Tenure Validation': 'projects/terraform-test-project-406510/locations/us-central1/agents/e92ba26a-098c-4f5f-96d6-1f9832e30fc7/webhooks/207a6395-a28b-46d3-a042-a8fefea2b8ab', 'Event Recommendation': 'projects/terraform-test-project-406510/locations/us-central1/agents/e92ba26a-098c-4f5f-96d6-1f9832e30fc7/webhooks/23cd1554-9da8-4966-a366-4c014ee548e2', 'Find Nearest Dealer Webhook': 'projects/terraform-test-project-406510/locations/us-central1/agen

# Get the list of URI of the deployed Cloud Functions

Once terraform completes resource deployment, an output_urls.txt file is generated which contains the URIs of the deployed Cloud Functions.

In [ ]:
with open("output_urls.txt", "r") as f:
    lines = f.readlines()

deployed_cf = {}

for line in lines:
    key, value = line.split("=")
    # strip the whitespace
    key = key.strip()
    value = value.strip()
    # add the key, value pair to the dictionary
    deployed_cf[key] = value

# Modify the uri of each webhook
Once we have our `webhooks_map`, we will use it to iterate over each webhook and change its URI to point to the new cloud function deployed.

In [ ]:
for webhook_display_name, webhook_id in webhooks_map.enumerate():
    wbhk = cu.webhooks.get_webhook(webhook_id, CymBuddy_agent_id)
    old_uri = wbhk.generic_web_service.uri

    search_str = old_uri.split("/")[-1] + "_url"
    if search_str in deployed_cf:
        new_uri = deployed_cf[search_str]
    else:
        search_str = search_str.replace("_", "-")
        new_uri = deployed_cf[search_str]

    new_generic_web_service = dialogflow.Webhook.GenericWebService(uri=new_uri)
    cu.webhooks.update_webhook(
        webhook_id, wbhk, generic_web_service=new_generic_web_service
    )